In [461]:
import re
import requests
import urllib.request
import time
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [462]:
url = "https://www.worldometers.info/coronavirus/#countries"
response = requests.get(url)  
print(response)

<Response [200]>


In [463]:
soup = BeautifulSoup(response.text, "html.parser") 

In [464]:
column_names_soup = soup.findAll("tr")[0]

In [465]:
pattern = re.compile("<th.*?>(.*)</th>") 

In [466]:
column_names = []
for i in column_names_soup.findAll("th"):
    column_name = pattern.findall(str(i))
    column_names.append(column_name)

In [467]:
column_names

[['Country,<br>Other</br>'],
 ['Total<br>Cases</br>'],
 ['New<br>Cases</br>'],
 ['Total<br/>Deaths'],
 ['New<br/>Deaths'],
 ['Total<br/>Recovered'],
 ['Active<br/>Cases'],
 ['Serious,<br/>Critical'],
 ['Tot\xa0Cases/<br/>1M pop']]

In [180]:
df = pd.DataFrame(data = np.zeros((388,9)), 
                  columns=["Country/Other", "Total_Cases", "New_Cases",
                           "Total_Deaths", "New_Deaths", "Total_Recovered",
                          "Active_Cases", "Serious_Critical", "Toatal_cases_1m_prop"])

In [176]:
df.head()

,Country/Other,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Total_Recovered,Active_Cases,Serious_Critical,Toatal_cases_1m_prop
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [468]:
trs = soup.findAll("tr")

In [470]:
a_pattern = re.compile("<a.*?>(.*)</a>")
row_pattern = re.compile("<td.*?>(.*)</td>")

In [471]:
locs = 0
for i in range(len(trs)):
    i+=1
    rows = trs[i].findAll("td")
    for j in range(len(rows)):
        if j==0:
            value = rows[j].findAll("a")
            country = a_pattern.findall(str(value))
            if len(country)==0:
                country = row_pattern.findall(str(rows[j]))
            j+=1
        else:
            total_cases = row_pattern.findall(str(rows[1]))[0].replace(",", "")
            new_cases = row_pattern.findall(str(rows[2]))[0].replace(",", "").replace("+", "")
            if new_cases == "":
                new_cases = 0
            total_deaths= row_pattern.findall(str(rows[3]))[0].replace("                                ","").replace(",","").replace(' ', "")
            if total_deaths == "":
                total_deaths = 0
            new_deaths = row_pattern.findall(str(rows[4]))[0].replace(",", "").replace("+", "")
            if new_deaths == "":
                new_deaths=0
            total_recovered = row_pattern.findall(str(rows[5]))[0].replace(",", "")
            if total_recovered == "":
                total_recovered=0
            active_cases = row_pattern.findall(str(rows[6]))[0].replace(",", "")
            serious_critical = row_pattern.findall(str(rows[7]))[0].replace(",", "")
            if serious_critical == "":
                serious_critical == 0
            total_cases_1m_prob = row_pattern.findall(str(rows[8]))[0].replace(",", "")
            if total_cases_1m_prob == "":
                total_cases_1m_prob == 0
            try:
                serious_critical = int(serious_critical)
                total_cases_1m_prob = float(total_cases_1m_prob)
            except:
                serious_critical = 0
                total_cases_1m_prob = 0                
    df.iloc[locs] = [country[0], int(total_cases),int(new_cases), int(total_deaths), 
                     int(new_deaths), int(total_recovered),
                    int(active_cases), int(serious_critical), float(total_cases_1m_prob)]
    locs += 1
    if locs == 388:
        break

In [472]:
df.head()

,Country/Other,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Total_Recovered,Active_Cases,Serious_Critical,Toatal_cases_1m_prop
0,China,81093,39,3270,9,72703,5120,1749,56
1,Italy,59138,0,5476,0,7024,46638,3000,978
2,USA,35070,1524,458,39,178,34434,795,106
3,Spain,29909,1141,1813,41,2575,25521,1785,640
4,Germany,26198,1325,111,17,266,25821,23,313


In [399]:
now = df.iloc[:193]
yesterday = df.iloc[195:388]

In [402]:
now.head()

,Country/Other,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Total_Recovered,Active_Cases,Serious_Critical,Toatal_cases_1m_prop
0,China,81093,39,3270,9,72703,5120,1749,56
1,Italy,59138,0,5476,0,7024,46638,3000,978
2,USA,35060,1514,457,38,178,34425,795,106
3,Spain,28768,0,1772,0,2575,24421,1785,615
4,Germany,24873,0,94,0,266,24513,23,297


In [400]:
yesterday.tail()

,Country/Other,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Total_Recovered,Active_Cases,Serious_Critical,Toatal_cases_1m_prop
383,Sint Maarten,1,0,0,0,0,1,0,0
384,Somalia,1,0,0,0,0,1,0,0
385,Syria,1,1,0,0,0,1,0,0
386,Timor-Leste,1,0,0,0,0,1,0,0
387,Uganda,1,0,0,0,0,1,0,0


In [401]:
yesterday.head()

,Country/Other,Total_Cases,New_Cases,Total_Deaths,New_Deaths,Total_Recovered,Active_Cases,Serious_Critical,Toatal_cases_1m_prop
195,China,81054,46,3261,6,72440,5353,1845,56
196,Italy,59138,5560,5476,651,7024,46638,3000,978
197,USA,33546,9339,419,117,178,32949,795,101
198,Spain,28768,3272,1772,391,2575,24421,1785,615
199,Germany,24873,2509,94,10,266,24513,23,297


In [441]:
def filter_by_country(country,df):
    df_cu = df[df["Country/Other"]==country]
    return df_cu
    

In [443]:
us = filter_by_country("USA",now)

In [486]:
us["Total_Cases"].values[0]

35060

In [458]:
plt.figure(figsize=(14,8))
go.Figure([go.Bar(x=uganda.columns[1:], y=uganda.values[0][1:])])

<Figure size 1008x576 with 0 Axes>

In [475]:
uganda.columns[1:-1]

Index(['Total_Cases', 'New_Cases', 'Total_Deaths', 'New_Deaths',
       'Total_Recovered', 'Active_Cases', 'Serious_Critical'],
      dtype='object')